<a href="https://colab.research.google.com/github/pavelpryadokhin/TTS/blob/main/%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_%D0%BF%D0%B5%D1%80%D0%B5%D0%B2%D0%BE%D0%B4%D0%B0_%D1%80%D0%B5%D1%87%D0%B8_%D0%B2_%D1%80%D0%B5%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%BC_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Система перевода речи в реальном времени

##Распознование языка

In [ ]:
!pip install -U pip
!pip install -q TTS  wget numpy==1.26.4 soundfile

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import numpy as np

class WhisperModel:
    def __init__(self, language="ru"):
        self.language = language
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self._load_model()

    def _load_model(self):
        model_name = "openai/whisper-small"
        self.processor = WhisperProcessor.from_pretrained(model_name)
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name).to(self.device)

        # Установка языка для генерации
        self.model.config.forced_decoder_ids = self.processor.get_decoder_prompt_ids(
            language=self.language,
            task="transcribe"
        )

    def transcribe_audio(self, audio):
        """
        Транскрибация аудио в реальном времени
        """
        audio_numpy, _ = librosa.load(audio, sr=16000, mono=True)
        # Подготовка входных данных
        inputs = self.processor(
            audio_numpy.astype(np.float32),
            sampling_rate=16000,
            return_tensors="pt",
            padding="max_length"
        ).input_features.to(self.device)

        # Генерация транскрипции
        predicted_ids = self.model.generate(inputs)

        # Декодирование результата
        transcription = self.processor.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription

# asr = WhisperModel()

##Перевод речи

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

class MarianMTTranslator:
    def __init__(self, language_pair):
        self.language_pair = language_pair
        self._initialize_model()

    def _initialize_model(self):
        model_mapping = {
            'ru_en': 'Helsinki-NLP/opus-mt-ru-en',
            'en_ru': 'Helsinki-NLP/opus-mt-en-ru',
            'fr_ru': 'Helsinki-NLP/opus-mt-fr-ru',
            'ru_fr': 'Helsinki-NLP/opus-mt-ru-fr',
            'fr_en': 'Helsinki-NLP/opus-mt-fr-en',
            'en_fr': 'Helsinki-NLP/opus-mt-en-fr'
        }

        if self.language_pair not in model_mapping:
            raise ValueError(f"Unsupported language pair: {self.language_pair}")
        model_name = model_mapping[self.language_pair]
        self.tokenizer = MarianTokenizer.from_pretrained(model_name)
        self.model = MarianMTModel.from_pretrained(model_name)

    def translate(self, text):
        inputs = self.tokenizer(text, return_tensors='pt',padding=True)
        translated_tokens = self.model.generate(**inputs)
        translated_text = self.tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

        return translated_text


# translator = MarianMTTranslator('ru_en')

##Клонирование речи

In [ ]:
import torch
from TTS.api import TTS
import librosa
import soundfile as sf

#устанавливаем зависимости
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.xtts_config import XttsConfig,XttsAudioConfig,XttsArgs
from torch.serialization import add_safe_globals
add_safe_globals([BaseDatasetConfig])
add_safe_globals([XttsConfig])
add_safe_globals([XttsAudioConfig])
add_safe_globals([XttsArgs])

class YourTTS:
    def __init__(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2",
                       progress_bar=False).to(device)

    def clone_voice(self, audio, text, language="en", output_path="output.wav"):
        try:
            # Генерация речи с клонированным голосом
            wav = self.tts.tts(
                text=text,
                speaker_wav=audio,
                language=language
            )
            # Сохранение результата
            sf.write(output_path, wav, self.tts.synthesizer.output_sample_rate)
            return True, "Голос успешно клонирован и сгенерирован"
        except Exception as e:
            return False,f"Ошибка при клонировании голоса: {str(e)}"

# tts = YourTTS()

##Система перевода

In [ ]:
class RealTimeTranslator:
    def __init__(self,source_lang, target_lang):
        """
        Инициализация системы перевода речи в реальном времени
        """
        self.source_lang = source_lang
        self.target_lang = target_lang

        # Инициализация компонентов
        self.asr = WhisperModel(source_lang)
        self.translator = MarianMTTranslator(f'{source_lang}_{target_lang}')
        self.tts = YourTTS()

    def process_audio_chunk(self, audio):
        """
        Обработка аудиофрагмента: распознавание -> перевод -> синтез речи
        """
        # Распознавание речи
        source_text = self.asr.transcribe_audio(audio)
        if not source_text.strip():
            return None

        # Перевод текста
        translated_text = self.translator.translate(source_text)

        # Синтез речи с сохранением характеристик голоса
        success, result = self.tts.clone_voice(
                audio=audio,
                text=translated_text,
                language=self.target_lang,
                output_path="temp_output.wav"
            )
        return success, result

    def analyze_voices(self, original_path, cloned_path):
        """
        Анализ характеристик голоса
        """
        y_orig, sr_orig = librosa.load(original_path, sr=None)
        y_clone, sr_clone = librosa.load(cloned_path, sr=None)

        # Извлечение MFCC-признаков
        mfcc_orig = librosa.feature.mfcc(y=y_orig, sr=sr_orig)
        mfcc_clone = librosa.feature.mfcc(y=y_clone, sr=sr_clone)

        # Выравнивание длины признаков
        min_frames = min(mfcc_orig.shape[1], mfcc_clone.shape[1])
        mfcc_orig = mfcc_orig[:, :min_frames]
        mfcc_clone = mfcc_clone[:, :min_frames]

        # Расчет корреляции между признаками
        correlation = np.corrcoef(mfcc_orig.flatten(), mfcc_clone.flatten())[0, 1]
        return correlation


##Тестирование

In [ ]:
from IPython.display import Audio
import time

#Доступка 3 языка на выбор: русский/английский/французский
path_audio_1 = 'test1.ogg'
path_audio_2 = 'test2.ogg'
source_lang = ['ru','en','fr']
target_lang = ['ru','en','fr']

translator = RealTimeTranslator(source_lang=source_lang[0], target_lang=target_lang[1])

def audio2audio(path_audio):
    start = time.perf_counter()
    success, result=translator.process_audio_chunk(path_audio)
    if not success:
        print(result)
    end = time.perf_counter()
    print(f"Выполнение заняло {end - start:.4f} сек")

    correlation = translator.analyze_voices(path_audio, 'temp_output.wav')
    print(f"\nКоэффициент корреляции между голосами: {correlation:.2f}")
    return Audio('temp_output.wav', rate=16000, autoplay=True)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


In [ ]:
# Оригинальная запись 1
Audio('test1.ogg', rate=16000, autoplay=True)

In [ ]:
# Переведенная запись 1
audio2audio(path_audio_1)

 > Text splitted to sentences.
["It's a test record for English translation."]
 > Processing time: 2.452507257461548
 > Real-time factor: 0.3882666931865819
Выполнение заняло 3.8320 сек

Коэффициент корреляции между голосами: 0.80


In [ ]:
# Оригинальная запись 2
Audio('test2.ogg', rate=16000, autoplay=True)

In [ ]:
# Переведенная запись 2
audio2audio(path_audio_2)

 > Text splitted to sentences.
["It's a test record to test the quality of the translation into a different language in real time.", 'It lasts 10 seconds.']
 > Processing time: 3.3765275478363037
 > Real-time factor: 0.39561953977741077
Выполнение заняло 6.0022 сек

Коэффициент корреляции между голосами: 0.81


#Вывод

Была разработана система перевода речи в реальном времени, которое:

* Распознает речь на входном языке
* Переводит на целевой язык
* Синтезирует речь на целевом языке
* Сохраняет характеристики голоса говорящего

Коэффициент корреляции между голосами составляет в среднем 80%